Import Modules

In [ ]:
import json
import requests
import time

import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys


Connexion API

In [ ]:
client_id = ''
client_secret = ''

client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)


Scrap playlists IDs

API function

In [ ]:
# Set category IDs
category_ids = ['0JQ5DAqbMKFQ00XGBls6ym', '0JQ5DAqbMKFEC4WFtoNRpw', '0JQ5DAqbMKFBweu2euf3Sj', '0JQ5DAqbMKFLVaM30PMBm4', '0JQ5DAudkNjCgYMM0TZXDr',
                '0JQ5DAqbMKFxXaXKP7zcDp', '0JQ5DAqbMKFzHmL4tf05da', '0JQ5DAqbMKFHOzuVTgTizF', '0JQ5DAqbMKFDXXwE9BDJAr', '0JQ5DAqbMKFCWjUTdzaG0e',
                '0JQ5DAqbMKFAXlCG6QvYQ4', '0JQ5DAqbMKFEZPnFQSFB1T', '0JQ5DAqbMKFNQ0fGp4byGU', '0JQ5DAqbMKFGvOw3O4nLAf', '0JQ5DAqbMKFFzDl7qN9Apr',
                '0JQ5DAqbMKFCuoRTxhYWow', '0JQ5DAqbMKFA6SOHvT3gck', '0JQ5DAqbMKFP9UNkavIGrM', '0JQ5DAqbMKFJKoGyUMo2hE', '0JQ5DAqbMKFx0uLQR2okcc',
                '0JQ5DAqbMKFIVNxQgRNSg0', '0JQ5DAqbMKFFtlLYUHv8bT', '0JQ5DAqbMKFAUsdyVjCQuL', '0JQ5DAqbMKFDkd668ypn6O', '0JQ5DAqbMKFAJ5xb0fwo9m',
                '0JQ5DAqbMKFQIL0AXnG5AK', '0JQ5DAqbMKFPw634sFwguI', '0JQ5DAqbMKFOOxftoKZxod', '0JQ5DAqbMKFPrEiAOxgac3', '0JQ5DAqbMKFy78wprEpAjl',
                '0JQ5DAqbMKFCbimwdOYlsl', '0JQ5DAqbMKFIpEuaCnimBj', '0JQ5DAqbMKFEOEBCABAxo9', '0JQ5DAqbMKFFoimhOqWzLB', '0JQ5DAqbMKFJw7QLnM27p6',
                '0JQ5DAqbMKFCfObibaOZbv', '0JQ5DAqbMKFLb2EqgLtpjC', '0JQ5DAqbMKFAjfauKLOZiv', '0JQ5DAqbMKFLjmiZRss79w', '0JQ5DAqbMKFKLfwjuJMoNC',
                '0JQ5DAqbMKFQiK2EHwyjcU', '0JQ5DAqbMKFQ1UFISXj59F', '0JQ5DAqbMKFAQy4HL4XU2D', '0JQ5DAqbMKFObNLOHydSW8', '0JQ5DAqbMKFSCjnQr8QZ3O',
                '0JQ5DAqbMKFI3pNLtYMD9S', '0JQ5DAqbMKFFsW9N8maB6z', '0JQ5DAqbMKFF9bY76LXmfI', '0JQ5DAqbMKFRKBHIxJ5hMm', '0JQ5DAqbMKFGnsSfvg90Wo',
                '0JQ5DAqbMKFLYQVFHcXMae', '0JQ5DAqbMKFDBgllo2cUIN', '0JQ5DAqbMKFOzQeOmemkuw', '0JQ5DAqbMKFziKOShCi009', '0JQ5DAqbMKFRieVZLLoo9m',
                '0JQ5DAqbMKFRY5ok2pxXJ0', '0JQ5DAqbMKFImHYGo3eTSg']

# Store missing categories
missing_categories = []

# Loop over all categories and get playlists
playlists = []
for category_id in category_ids:
    try:
        # Get first 50 playlists in category
        response = sp.category_playlists(category_id, limit=50)
        category_playlists = response["playlists"]["items"]
        
        # Check if there are more playlists in category
        while response["playlists"]["next"]:
            response = sp.next(response["playlists"])
            category_playlists += response["playlists"]["items"]
        
        # Append category playlists to list
        for playlist in category_playlists:
            playlist_id = playlist["id"]
            playlist_name = playlist["name"]
            playlists.append({"category_id": category_id, "playlist_id": playlist_id, "playlist_name": playlist_name})
    except: 
        missing_categories.append(category_id)
        print(f"Category with ID {category_id} doesn't exist.")

# Create dataframe from playlists
df_playlists = pd.DataFrame(playlists, columns=["category_id", "playlist_id", "playlist_name"])

print(df_playlists)


HTML scrapping for missing categories

In [ ]:
urls = [
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1h',
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1i',
    'https://open.spotify.com/genre/section0JQ5DAzQHECxDlYNI6xD1j'
]

# Set up the web driver
driver = webdriver.Safari(executable_path='/usr/bin/safaridriver')

top_playlists = []

for url in urls:
    driver.get(url)

    # Wait for the playlists to load
    time.sleep(5)  # Wait for initial playlists to load

    playlists = driver.find_elements(By.CLASS_NAME, 'Nqa6Cw3RkDMV8QnYreTr')
    last_playlist = playlists[-1]

    while True:
        # Scroll down to the last playlist
        driver.execute_script("arguments[0].scrollIntoView();", last_playlist)
        time.sleep(2)
        # Check if there are new playlists
        new_playlists = driver.find_elements(By.CLASS_NAME, 'Nqa6Cw3RkDMV8QnYreTr')
        if new_playlists[-1] != last_playlist:
            last_playlist = new_playlists[-1]
            playlists = new_playlists
        else:
            break

    # Extract playlist information
    for playlist in playlists:
        href = playlist.get_attribute('href')
        playlist_id = href.split('/')[-1]
        playlist_name = playlist.text
        top_playlists.append({"category_id": '0JQ5DAudkNjCgYMM0TZXDr', "playlist_id": playlist_id, "playlist_name": playlist_name})

# Quit the driver
driver.quit()

df_top_playlists = pd.DataFrame(top_playlists, columns=["category_id", "playlist_id", "playlist_name"])

print(df_top_playlists)


Parse playlists info into excel

In [ ]:
# Combine the manual and API playlists into one DataFrame
df_combined = pd.concat([df_playlists, df_top_playlists], ignore_index=True)

# Export the combined DataFrame to an Excel file
df_combined.to_excel("playlists_info.xlsx", index=False)

# Drop duplicates based on the "playlist_id" column, keeping the first occurrence
df_cleaned = df_combined.drop_duplicates(subset='playlist_id', keep='first')

# Export the cleaned DataFrame to an Excel file
df_cleaned.to_excel("playlists_info_clean.xlsx", index=False)


Request data and create dataset

In [ ]:
df = pd.read_excel('playlists_info_clean.xlsx')


Define scrapping functions

In [ ]:
# Define a function to get all the tracks in a playlist, handling the API limit
def get_all_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

# Define a function to get all the audio features for a set of tracks, handling the API limit
def get_all_audio_features(track_ids):
    audio_features = []
    for i in range(0, len(track_ids), 100):
        batch = track_ids[i:i+100]
        try:
            features = sp.audio_features(batch)
        except:
            continue
        audio_features.extend(features)
    return audio_features

def get_playlist_track_info(playlist_id):
    # Get the playlist info
    playlist_info = sp.playlist(playlist_id)
    if not playlist_info:
        return None
    playlist_name = playlist_info['name']
    playlist_followers = playlist_info['followers']['total']
    
    # Get all the tracks in the playlist
    playlist_tracks = get_all_playlist_tracks(playlist_id)
    track_ids = [track['track']['id'] for track in playlist_tracks if track['track'] is not None]
    
    if not track_ids:
        return None
    
    # Get the track info in batches of 50
    track_info = []
    album_ids = []  # Store the album IDs for later retrieval
    for i in range(0, len(track_ids), 50):
        batch_tracks = track_ids[i:i+50]
        tracks = sp.tracks(batch_tracks)['tracks']
        track_info.extend(tracks)
        album_ids.extend([track['album']['id'] for track in tracks])
    
    track_names = [track['name'] for track in track_info]
    track_popularities = [track['popularity'] for track in track_info]

    artist_ids = [track['artists'][0]['id'] for track in track_info]
    artist_info = []
    for i in range(0, len(artist_ids), 50):
        batch_artists = artist_ids[i:i+50]
        artists = sp.artists(batch_artists)['artists']
        artist_info.extend(artists)
    
    artist_names = [artist['name'] for artist in artist_info]
    artist_popularities = [artist['popularity'] for artist in artist_info]
    
    # Get the audio features
    audio_features = get_all_audio_features(track_ids)
    if audio_features is None:
        return None

    # Get the album info
    album_info = []
    for i in range(0, len(album_ids), 20):
        batch_albums = album_ids[i:i+20]
        albums = sp.albums(batch_albums)['albums']
        album_info.extend(albums)
    
    labels = [album['label'] for album in album_info]
    if labels is None:
        return None

    # Combine everything into a DataFrame
    playlist_df = pd.DataFrame({
        'playlist_name': playlist_name,
        'playlist_followers': playlist_followers,
        'track_name': track_names,
        'popularity_track': track_popularities,
        'artist_name': artist_names,
        'popularity_artist': artist_popularities,
        'label': labels  # Add the label column
    })
    
    # Add the audio features to the DataFrame
    try:
        audio_features_df = pd.DataFrame(audio_features)
        playlist_df = pd.concat([playlist_df, audio_features_df], axis=1)
    except:
        return None
    
    return playlist_df

# Define a function to loop over all the playlists and combine the results
def get_all_playlist_track_info(playlist_ids):
    playlist_dfs = []
    for playlist_id in playlist_ids:
        playlist_df = get_playlist_track_info(playlist_id)
        if playlist_df is not None:
            playlist_dfs.append(playlist_df)
    all_playlist_df = pd.concat(playlist_dfs, ignore_index=True)
    return all_playlist_df


Scrap the data for each category and parse into excel

In [ ]:
category_id = "0JQ5DAqbMKFQ00XGBls6ym" # Rap

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_rap = get_all_playlist_track_info(playlist_ids)

df_rap.to_excel("df_rap.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFEC4WFtoNRpw" # Pop

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_pop = get_all_playlist_track_info(playlist_ids)

df_pop.to_excel("df_pop.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFBweu2euf3Sj" # Variété

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_variete = get_all_playlist_track_info(playlist_ids)

df_variete.to_excel("df_variete.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFLVaM30PMBm4" # Eté

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ete = get_all_playlist_track_info(playlist_ids)

df_ete.to_excel("df_ete.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFxXaXKP7zcDp" # Latino

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_latino = get_all_playlist_track_info(playlist_ids)

df_latino.to_excel("df_latino.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFzHmL4tf05da" # Ambiance

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ambiance = get_all_playlist_track_info(playlist_ids)

df_ambiance.to_excel("df_ambiance.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFHOzuVTgTizF" # Electro

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_electro = get_all_playlist_track_info(playlist_ids)

df_electro.to_excel("df_electro.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFDXXwE9BDJAr" # Rock

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_rock = get_all_playlist_track_info(playlist_ids)

df_rock.to_excel("df_rock.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFCWjUTdzaG0e" # Indie

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_indie = get_all_playlist_track_info(playlist_ids)

df_indie.to_excel("df_indie.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAXlCG6QvYQ4" # Sport

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_sport = get_all_playlist_track_info(playlist_ids)

df_sport.to_excel("df_sport.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFEZPnFQSFB1T" # RnB

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_rnb = get_all_playlist_track_info(playlist_ids)

df_rnb.to_excel("df_rnb.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFNQ0fGp4byGU" # Afro

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_afro = get_all_playlist_track_info(playlist_ids)

df_afro.to_excel("df_afro.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFGvOw3O4nLAf" # K-pop

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_kpop = get_all_playlist_track_info(playlist_ids)

df_kpop.to_excel("df_kpop.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFFzDl7qN9Apr" # Détente

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_detente = get_all_playlist_track_info(playlist_ids)

df_detente.to_excel("df_detente.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFCuoRTxhYWow" # Dormir

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_dormir = get_all_playlist_track_info(playlist_ids)

df_dormir.to_excel("df_dormir.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFA6SOHvT3gck" # Soirée

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_soiree = get_all_playlist_track_info(playlist_ids)

df_soiree.to_excel("df_soiree.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFP9UNkavIGrM" # Zouk

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_zouk = get_all_playlist_track_info(playlist_ids)

df_zouk.to_excel("df_zouk.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFJKoGyUMo2hE" # Reggae

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_reggae = get_all_playlist_track_info(playlist_ids)

df_reggae.to_excel("df_reggae.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFx0uLQR2okcc" # A la maison

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_maison = get_all_playlist_track_info(playlist_ids)

df_maison.to_excel("df_maison.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFIVNxQgRNSg0" # Decennies

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_decennies = get_all_playlist_track_info(playlist_ids)

df_decennies.to_excel("df_decennies.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFFtlLYUHv8bT" # Alternatif

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_alternatif = get_all_playlist_track_info(playlist_ids)

df_alternatif.to_excel("df_alternatif.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAUsdyVjCQuL" # Amour

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_amour = get_all_playlist_track_info(playlist_ids)

df_amour.to_excel("df_amour.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFDkd668ypn6O" # Metal

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_metal = get_all_playlist_track_info(playlist_ids)

df_metal.to_excel("df_metal.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAJ5xb0fwo9m" # Jazz

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_jazz = get_all_playlist_track_info(playlist_ids)

df_jazz.to_excel("df_jazz.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFQIL0AXnG5AK" # Tendances

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tendances = get_all_playlist_track_info(playlist_ids)

df_tendances.to_excel("df_tendances.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFPw634sFwguI" # Equal

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_equal = get_all_playlist_track_info(playlist_ids)

df_equal.to_excel("df_equal.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFPrEiAOxgac3" # Classique

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_classique = get_all_playlist_track_info(playlist_ids)

df_classique.to_excel("df_classique.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFy78wprEpAjl" # Folk

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_folk = get_all_playlist_track_info(playlist_ids)

df_folk.to_excel("df_folk.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFCbimwdOYlsl" # Focus

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_focus = get_all_playlist_track_info(playlist_ids)

df_focus.to_excel("df_focus.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFIpEuaCnimBj" # Soul

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_soul = get_all_playlist_track_info(playlist_ids)

df_soul.to_excel("df_soul.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFEOEBCABAxo9" # Netflix

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_netflix = get_all_playlist_track_info(playlist_ids)

df_netflix.to_excel("df_netflix.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFFoimhOqWzLB" # Enfants

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_enfants = get_all_playlist_track_info(playlist_ids)

df_enfants.to_excel("df_enfants.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFJw7QLnM27p6" # Etudiants

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_etudiants = get_all_playlist_track_info(playlist_ids)

df_etudiants.to_excel("df_etudiants.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFLb2EqgLtpjC" # Bien-être

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_bienetre = get_all_playlist_track_info(playlist_ids)

df_bienetre.to_excel("df_bienetre.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAjfauKLOZiv" # Punk

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_punk = get_all_playlist_track_info(playlist_ids)

df_punk.to_excel("df_punk.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFLjmiZRss79w" # Ambient

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_ambient = get_all_playlist_track_info(playlist_ids)

df_ambient.to_excel("df_ambient.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFKLfwjuJMoNC" # Country

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_country = get_all_playlist_track_info(playlist_ids)

df_country.to_excel("df_country.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFQiK2EHwyjcU" # Blues

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_blues = get_all_playlist_track_info(playlist_ids)

df_blues.to_excel("df_blues.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFQ1UFISXj59F" # Arabe

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_arabe = get_all_playlist_track_info(playlist_ids)

df_arabe.to_excel("df_arabe.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFAQy4HL4XU2D" # Voyage

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_voyage = get_all_playlist_track_info(playlist_ids)

df_voyage.to_excel("df_voyage.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAqbMKFF9bY76LXmfI" # Frequency

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_frequency = get_all_playlist_track_info(playlist_ids)

df_frequency.to_excel("df_frequency.xlsx", index = False)


In [ ]:
category_id = "0JQ5DAudkNjCgYMM0TZXDr" # Tops

filtered_df = df[df['category_id'] == category_id]
playlist_ids = filtered_df['playlist_id'].tolist()

df_tops = get_all_playlist_track_info(playlist_ids)

df_tops.to_excel("df_tops.xlsx", index = False)
